In [1]:
# Package Loading

from datetime import datetime as dt
import time
from selenium import webdriver
from selenium.webdriver.support.select import Select
import pandas as pd

In [20]:
# Driver Initializing

driver = webdriver.Chrome()

In [21]:
# Browser Opening

driver.get('https://www.booking.com/searchresults.en-us.html?ss=Da+Lat&ssne=Da+Lat&ssne_untouched=Da+Lat&efdco=1&label=a467b9b2feda11eda9b21926e8cdcb9d&sid=cac42c0c6d9b8ee1a143c8275873f698&aid=2167732&lang=en-us&sb=1&src_elem=sb&src=searchresults&dest_id=-3712045&dest_type=city&checkin=2023-06-01&checkout=2023-06-02&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure')
driver.maximize_window()
time.sleep(3)

In [22]:
# Page Selecting

def page_picker(page_xpath):
    driver.find_element('xpath', page_xpath).click()
    time.sleep(5)

In [23]:
# Data scraping

def data_scraping(accommodation_list, original_price_list, discounted_price_list, star_list, distance_list, room_list, rate_list, rcount_list):
    
    cards = driver.find_elements('xpath', '//div[@class="b978843432"]')
    
    # Accommodation
    
    accommodations = driver.find_elements('xpath','//div[@data-testid="title"]')

    for accommodation in accommodations:
        accommodation_list.append(accommodation.text)
    
    # Original Price

    for price in cards:
        child_original_prices = price.find_elements('xpath', './/span[@class="c5888af24f e729ed5ab6"]')
        child_original_price_list = []
        for child_original_price in child_original_prices:
            child_original_price_list.append(child_original_price.text)
        original_price_list.append(child_original_price_list)
    
    # Discounted Price
    
    discounted_prices = driver.find_elements('xpath','//span[@data-testid="price-and-discounted-price"]')

    for price in discounted_prices:
        discounted_price_list.append(price.text)
    
    # Star
    
    for star in cards:
        child_stars = star.find_elements('xpath', './/div[@class="e4755bbd60"]')
        child_star_list = []
        for child_star in child_stars:
            child_star_list.append(child_star.get_attribute("aria-label"))
        star_list.append(child_star_list)
    
    # Distance to Center
    
    distances = driver.find_elements('xpath','//span[@data-testid="distance"]')

    for distance in distances:
        distance_list.append(distance.text)
    
    # Room

    for room in cards:
        child_rooms = room.find_elements('xpath', './/span[@class="df597226dd"]')
        child_room_list = []
        for child_room in child_rooms:
            child_room_list.append(child_room.text)
        room_list.append(child_room_list)
        
    # Rate

    for rate in cards:
        child_rates = rate.find_elements('xpath', './/div[@class="b5cd09854e d10a6220b4"]')
        child_rate_list = []
        for child_rate in child_rates:
            child_rate_list.append(child_rate.text)
        rate_list.append(child_rate_list)
    
    # Rate Count
    
    for rcount in cards:
        child_rcounts = rcount.find_elements('xpath', './/div[@class="d8eab2cf7f c90c0a70d3 db63693c62"]')
        child_rcount_list = []
        for child_rcount in child_rcounts:
            child_rcount_list.append(child_rcount.text)
        rcount_list.append(child_rcount_list)
    
    return accommodation_list, original_price_list, discounted_price_list, distance_list, room_list, rate_list, rcount_list

In [24]:
# Main Procedure

accommodation_list, original_price_list, discounted_price_list, star_list, distance_list, room_list, rate_list, rcount_list = ([] for _ in range(8))

# Page Selection

pages = driver.find_elements('xpath','//button[@class="fc63351294 f9c5690c58"]')
page_counts = []

for page in pages:
    page_counts.append(int(page.text))

page_xpaths = []

for i in range(page_counts[-1]):
    page_xpaths.append('//button[@aria-label=' + str(i+1) + ']')

for page_xpath in page_xpaths:
    page_picker(page_xpath)
    data_scraping(accommodation_list, original_price_list, discounted_price_list, star_list, distance_list, room_list, rate_list, rcount_list)

In [25]:
# Dataset Creating

df = pd.DataFrame({'accommodation_name': accommodation_list, 'original_price': original_price_list, 'discounted_price': discounted_price_list, 'star_rating': star_list, 'room_type': room_list, 'distance_to_center': distance_list, 'rate': rate_list, 'rate_count': rcount_list})
df

,accommodation_name,original_price,discounted_price,star_rating,room_type,distance_to_center,rate,rate_count
0,The Morris Hotel Da Lat,"[VND 800,000]","VND 400,000",[3 out of 5],[Standard Double Room],1.3 km from center,[8.8],[177 reviews]
1,Dalat Wind Hotel,"[VND 554,400]","VND 415,800",[2 out of 5],[Economy Quadruple Room - Basement],0.7 km from center,[8.5],[397 reviews]
2,Bazan Hotel Dalat,"[VND 750,000]","VND 412,500",[3 out of 5],[Standard Double Room],1.8 km from center,[9.1],"[1,229 reviews]"
3,Nhà nghỉ THANH TÙNG,"[VND 270,000]","VND 243,000",[],[Double Room],1 km from center,[],[]
4,HANZ Bao Hoang Hotel 48 Ha Huy Tap,"[VND 487,200]","VND 263,088",[3 out of 5],[Standard Queen Room],0.9 km from center,[6.8],[36 reviews]
...,...,...,...,...,...,...,...,...
984,Thiên Thanh,[],"VND 702,000",[],[Double Room],1.9 km from center,[8.7],[35 reviews]
985,New Sleep in Dalat Hostel,"[VND 392,400]","VND 353,160",[],[Bunk Bed in Mixed Dormitory Room],0.6 km from center,[9.6],[29 reviews]
986,The Mountain Homestay Đà Lạt,[],"VND 920,000",[],"[King Room with Balcony, Double Room]",1.6 km from center,[],[]
987,Tashi House Dalat (Thong Nho oi),[],"VND 280,000",[],[Single Bed in 4-Bed Dormitory Room],350 m from center,[8.3],[4 reviews]


In [26]:
# Data Transforming

def original_price_func(x):
    try:
        return float(x[0][4:].replace(',',''))
    except IndexError:
        return 'null'

original_price_list_v2 = [original_price_func(x) for x in original_price_list]

def discounted_price_func(x):
    try:
        return float(x[4:].replace(',',''))
    except IndexError:
        return 'null'

discounted_price_list_v2 = [discounted_price_func(x) for x in discounted_price_list]

def star_func(x):
    try:
        return x[0][0]
    except IndexError:
        return 'null'

star_list_v2 = [star_func(x) for x in star_list]

def room_func(x):
    try:
        return x[0]
    except IndexError:
        return 'null'

room_list_v2 = [room_func(x) for x in room_list]

def distance_func(x):
    try:
        return float(x[:-15])
    except IndexError:
        return 'null'

distance_list_v2 = [distance_func(x) for x in distance_list]

def rate_func(x):
    try:
        return float(x[0])
    except IndexError:
        return 'null'

rate_list_v2 = [rate_func(x) for x in rate_list]

def rcount_func(x):
    try:
        return int(x[0][:x[0].find(' ')].replace(',', ''))
    except IndexError:
        return 'null'

rcount_list_v2 = [rcount_func(x) for x in rcount_list]

In [27]:
# Dataset Recreating

df = pd.DataFrame({'accommodation_name': accommodation_list, 'original_price': original_price_list_v2, 'discounted_price': discounted_price_list_v2, 'star_rating': star_list_v2, 'room_type': room_list_v2, 'distance_to_center': distance_list_v2, 'rate': rate_list_v2, 'rate_count': rcount_list_v2})
df

,accommodation_name,original_price,discounted_price,star_rating,room_type,distance_to_center,rate,rate_count
0,The Morris Hotel Da Lat,800000.0,400000.0,3,Standard Double Room,1.3,8.8,177
1,Dalat Wind Hotel,554400.0,415800.0,2,Economy Quadruple Room - Basement,0.7,8.5,397
2,Bazan Hotel Dalat,750000.0,412500.0,3,Standard Double Room,1.8,9.1,1229
3,Nhà nghỉ THANH TÙNG,270000.0,243000.0,null,Double Room,1.0,null,null
4,HANZ Bao Hoang Hotel 48 Ha Huy Tap,487200.0,263088.0,3,Standard Queen Room,0.9,6.8,36
...,...,...,...,...,...,...,...,...
984,Thiên Thanh,null,702000.0,null,Double Room,1.9,8.7,35
985,New Sleep in Dalat Hostel,392400.0,353160.0,null,Bunk Bed in Mixed Dormitory Room,0.6,9.6,29
986,The Mountain Homestay Đà Lạt,null,920000.0,null,King Room with Balcony,1.6,null,null
987,Tashi House Dalat (Thong Nho oi),null,280000.0,null,Single Bed in 4-Bed Dormitory Room,35.0,8.3,4


In [28]:
df.to_excel('booking_dataset.xlsx')